# Importing the databases

In [ ]:
import pandas as pd

df = pd.read_csv("../data/df_final_demo.txt")
final_demo_df = df.copy()

final_demo_df

In [ ]:
import pandas as pd

df = pd.read_csv("../data/df_final_web_data_pt_1.txt")
final_web_data_1_df = df.copy()

final_web_data_1_df

In [ ]:
import pandas as pd

df = pd.read_csv("../data/df_final_web_data_pt_2.txt")
final_web_data_2_df = df.copy()

final_web_data_2_df

In [ ]:
import pandas as pd

df = pd.read_csv("../data/df_final_experiment_clients.txt")
experiment_clients_df = df.copy()

experiment_clients_df

# Sanity check of the databases

### Important notes

Things to pay attention to while merging:
- Make sure that reach client_id is either in the control group, or the test group, but not both.
- One client_id can have multiple visitor_id's, but not the other way around. Visitor_id should have no duplicates.
- We merge dataframes ON client_id, which is the common denominator column of the three tables.

How to define the time spent per step:
- Each row provide the timestamp of the client initiating a step
- therefore, we will need to group per client_id and sort the values of time in an natural order
- Then we need to create two more columns, one for "duration", one for "success" - a boolean column that specifies whether the client proceeded or did a step back (error) at each step.
- Lastly, we need to create a column for our most important KPI, which is "conversion". That means, that a customer has proceeded in all steps and finalized the confirmation.

However, here are some important biases we need to account for:
- Session fragmentation -> Create a column for "SESSIONS" per client

A single client producing several visitor_id values within the same experiment window may represent broken sessions rather than distinct attempts. The simple check is: count visitor_id per client_id. If most clients have one and a few have many, inspect their time ordering. If multiple visitor_id values overlap in time, they likely represent a single attempt. A simple rule:
If two visitor_ids within the same client_id occur less than ~5 minutes apart and both start at step 1, you can treat them as the same attempt. If not, leave them separate.

- Inconsistent step ordering.

For each client_id, sort by date_time and verify that process_step never jumps backward by more than one. Small backward moves usually indicate page refreshes. Large jumps indicate noise.
We should flag sequences where process_step is not monotonically increasing and either exclude them or report them as noisy.

- Temporal truncation.

Our experiment has a fixed end date. Any visit_id whose final event occurs near that boundary might not have had time to finish. Compute the time difference between the last observed step and the experiment end. If the gap is very small, treat the session as incomplete by truncation rather than failure. You can either exclude them or keep them but acknowledge the ambiguity.

- Arm misclassification.

Each client_id appears exactly once and in exactly one group in the experiment file. If duplicates appear or if a client_id in the web logs is missing from the experiment file, flag and exclude.




### Step 1: Experiment data - Load and sanity-check each table.

Verify row counts, missing client_id, duplicate client_id in the experiment roster, and duplicate visitor_id in the web logs. This establishes whether the dataset is even suitable for merging.

### Step 2: Experiment data - Validate absence of arm misclassification.

Each client_id appears exactly once and in exactly one group in the experiment file. If duplicates appear or if a client_id in the web logs is missing from the experiment file, flag and exclude.


In [ ]:
#Check if there are any duplicates in experiment dataFrame
experiment_clients_df[experiment_clients_df.duplicated()]

In [ ]:
#Check the unique vlaues in the experiment dataframe
experiment_clients_df["client_id"].nunique()

In [ ]:
#Check the unique vlaues in the web data 1 dataframe
final_web_data_1_df["client_id"].nunique()

In [ ]:
#Check the unique vlaues in the web data 1 dataframe
final_web_data_2_df["client_id"].nunique()

In [ ]:
#Combined the web data 1 and web data 2 data frames 
combined_web_data = (
    pd.concat([final_web_data_1_df, final_web_data_2_df])
    .drop_duplicates(subset="client_id", keep = "first")
    .reset_index(drop=True)
)
combined_web_data

In [ ]:
#Get the client ids only in combined data frame but not present in experiment data frame
ids_only_in_df1 = combined_web_data.loc[
    ~combined_web_data["client_id"].isin(experiment_clients_df["client_id"]),
    "client_id"
].unique()
ids_only_in_df1

### Step 3: Web logs data - Inspect session multiplicity

Group web logs by client_id and count distinct visitor_id. If most clients have one and a minority have many, keep visitor_id as the unit of analysis. Only collapse visitor_id when two IDs begin a step-1 sequence within minutes of each other.

### Step 4: Web logs data - Account for Session Fragmentation - Build "session" timelines

For each (client_id, visitor_id), sort by date_time. Check monotonicity of process_step. Minor regressions can be tolerated; major reversals get flagged and excluded

### Step 5: Web logs data - Compute step durations and success-level outcomes.

Within each sorted sequence, compute time between steps, derive a success flag per step, and classify the visitor_id as converted or not.

### Step 6: Temporal truncation

Any attempt whose last timestamp sits very close to the experiment’s end can be marked ambiguous. We may exclude them or keep them with a clear note that their status is censored.

# Merging the databases

Merge on client_id, using the experiment client_id as the base, since these are the only customers that matter.

We prefer "left" merge instead of "inner" because:
how="left" keeps all experiment clients, including:
- those who never visited (web columns NaN)
- those missing demographics (demo columns NaN)

An inner merge would silently drop:
- assigned clients with no web activity
- assigned clients missing demographics

For an A/B test, dropping assigned-but-inactive clients biases completion rates, so left is preferable. We can always later filter to “clients with web activity and complete demographics” explicitly, instead of letting the join hide them.

In [ ]:
 # exp_demo = experiment.merge(demo, on="client_id", how="left")

# then

# web = pd.concat([web_pt1, web_pt2], ignore_index=True)

# and lastly

# full = exp_demo.merge(web_data, on="client_id", how="left")
